In [1]:
# Attempt to rollup most of NSE programs into ONE

# STATUS: Incomplete (Trial with index options)
# Run-time: Unknown

# Dependencies:
#  - List of indexes and equities
#  - underlying's ohlc history for standard deviation and Black Scholes

#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
ib = IB().connect('127.0.0.1', 7496, clientId=5) # kavi tws live
# ib = IB().connect('127.0.0.1', 4001, clientId=5) # kavi IBG live
# ib = IB().connect('127.0.0.1', 7499, clientId=5) # kavi tws paper-trade
# ib = IB().connect('127.0.0.1', 4004, clientId=5) # kavi IBG paper-trade

In [2]:
%%time
#***      Imports    *****
import pandas as pd
import numpy as np
from itertools import product
import utils

#***    Paths and Variables    ***
datapath = r'./zdata/'
market = 'NSE'
deletesym = ['USDINR']   #ibSymbols to be deleted

#***    Get scrips from IBKR   ***
df_ib = pd.DataFrame([]) # Initialize

# from IBKR's symbols page
url = "https://www.interactivebrokers.com.hk/en/index.php?"
url = url+"f=2222&exch=nse&showcategories=OPTGRP&page="

for i in range(4):
    df_ib = df_ib.append(df_ib.append(pd.read_html(url+str(i), header=0)[2]))

# Exit program if dataframe is empty
if df_ib.empty:
    print('Interactive Brokers dataframe is empty!!!')
    sys.exit(0)
    
df_ib = df_ib.drop_duplicates().reset_index(drop=True)
all_symbols = list(df_ib['IB Symbol'])

# Qualify Stock contracts
stocks = [Stock(symbol=s, exchange=market) for s in all_symbols]
stock_contracts = ib.qualifyContracts(*stocks)
stock_symbols = [s.symbol for s in stock_contracts]

# Qualify Index contracts
idx_symbols = [a for a in all_symbols if a not in stock_symbols]

indexes = [Index(symbol=s, exchange='NSE') for s in idx_symbols]
index_contracts = ib.qualifyContracts(*indexes)
index_symbols = [i.symbol for i in index_contracts]

# All qualified symbols and contracts
symbols = stock_symbols + index_symbols
contracts = stock_contracts + index_contracts

ERROR:ib_insync.wrapper:Error 200, reqId 2651: No security definition has been found for the request, contract: Stock(symbol='BANKNIFTY', exchange='NSE')
ERROR:ib_insync.wrapper:Error 200, reqId 2765: No security definition has been found for the request, contract: Stock(symbol='NIFTY50', exchange='NSE')
ERROR:ib_insync.wrapper:Error 200, reqId 2826: No security definition has been found for the request, contract: Stock(symbol='USDINR', exchange='NSE')
ERROR:ib_insync.ib:Unknown contract: Stock(symbol='BANKNIFTY', exchange='NSE')
ERROR:ib_insync.ib:Unknown contract: Stock(symbol='NIFTY50', exchange='NSE')
ERROR:ib_insync.ib:Unknown contract: Stock(symbol='USDINR', exchange='NSE')


Wall time: 22.7 s


In [ ]:
#****     DATA LIMITER HERE   ****
#_________________________________
# symbols = stock_symbols + index_symbols # DATA LIMITER!!
# contracts = stock_contracts + index_contracts # DATA LIMITER!!

In [ ]:
# import importlib
# importlib.reload(utils)

# [utils.getprice(contract=c, ib=ib) 
#  for i in range(0, len(contracts), blk) 
#  for c in contracts[i:i+blk]]

In [3]:
blk = 20
optprice = [(c.symbol, utils.getprice(contract=c, ib=ib)) 
for i in range(0, len(contracts), blk) 
for c in contracts[i:i+blk]]

ERROR:ib_insync.wrapper:Error 322, reqId 2887: Error processing request:-'bN' : cause - Only 50 simultaneous API historical data requests allowed., contract: Stock(conId=56984045, symbol='DCBBANK', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='DCBBANK', tradingClass='DCBBANK')
ERROR:ib_insync.wrapper:Error 322, reqId 2888: Error processing request:-'bN' : cause - Only 50 simultaneous API historical data requests allowed., contract: Stock(conId=56984051, symbol='DHFL', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='DHFL', tradingClass='DHFL')
ERROR:ib_insync.wrapper:Error 322, reqId 2889: Error processing request:-'bN' : cause - Only 50 simultaneous API historical data requests allowed., contract: Stock(conId=56984081, symbol='DISHTV', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='DISHTV', tradingClass='DISHTV')
ERROR:ib_insync.wrapper:Error 322, reqId 2890: Error processing request:-'bN' : cause - Only 50 simultaneous API

ERROR:ib_insync.wrapper:Error 322, reqId 2915: Error processing request:-'bN' : cause - Only 50 simultaneous API historical data requests allowed., contract: Stock(conId=44652060, symbol='HINDUNILV', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='HINDUNILVR', tradingClass='HINDUNILV')
ERROR:ib_insync.wrapper:Error 322, reqId 2916: Error processing request:-'bN' : cause - Only 50 simultaneous API historical data requests allowed., contract: Stock(conId=84948060, symbol='HINDZINC', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='HINDZINC', tradingClass='HINDZINC')
ERROR:ib_insync.wrapper:Error 322, reqId 2917: Error processing request:-'bN' : cause - Only 50 simultaneous API historical data requests allowed., contract: Stock(conId=125585152, symbol='IBULHSGFI', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='IBULHSGFIN', tradingClass='IBULHSGFI')
ERROR:ib_insync.wrapper:Error 322, reqId 2918: Error processing request:-'bN' : ca

ERROR:ib_insync.wrapper:Error 322, reqId 2943: Error processing request:-'bN' : cause - Only 50 simultaneous API historical data requests allowed., contract: Stock(conId=250071823, symbol='KAJARIACE', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='KAJARIACER', tradingClass='KAJARIACE')
ERROR:ib_insync.wrapper:Error 322, reqId 2944: Error processing request:-'bN' : cause - Only 50 simultaneous API historical data requests allowed., contract: Stock(conId=78743661, symbol='KOTAKBANK', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='KOTAKBANK', tradingClass='KOTAKBANK')
ERROR:ib_insync.wrapper:Error 322, reqId 2945: Error processing request:-'bN' : cause - Only 50 simultaneous API historical data requests allowed., contract: Stock(conId=56985099, symbol='KPIT', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='KPIT', tradingClass='KPIT')
ERROR:ib_insync.wrapper:Error 322, reqId 2946: Error processing request:-'bN' : cause - Only 50

ERROR:ib_insync.wrapper:Error 322, reqId 2971: Error processing request:-'bN' : cause - Only 50 simultaneous API historical data requests allowed., contract: Stock(conId=56984634, symbol='NIITTECH', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='NIITTECH', tradingClass='NIITTECH')
ERROR:ib_insync.wrapper:Error 322, reqId 2972: Error processing request:-'bN' : cause - Only 50 simultaneous API historical data requests allowed., contract: Stock(conId=56984653, symbol='NMDC', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='NMDC', tradingClass='NMDC')
ERROR:ib_insync.wrapper:Error 322, reqId 2973: Error processing request:-'bN' : cause - Only 50 simultaneous API historical data requests allowed., contract: Stock(conId=49982069, symbol='NTPC', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='NTPC', tradingClass='NTPC')
ERROR:ib_insync.wrapper:Error 322, reqId 2974: Error processing request:-'bN' : cause - Only 50 simultaneous API hi

ERROR:ib_insync.wrapper:Error 322, reqId 2998: Error processing request:-'bN' : cause - Only 50 simultaneous API historical data requests allowed., contract: Stock(conId=44652045, symbol='SAIL', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='SAIL', tradingClass='SAIL')
ERROR:ib_insync.wrapper:Error 322, reqId 2999: Error processing request:-'bN' : cause - Only 50 simultaneous API historical data requests allowed., contract: Stock(conId=173418084, symbol='SBIN', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='SBIN', tradingClass='SBIN')
ERROR:ib_insync.wrapper:Error 322, reqId 3000: Error processing request:-'bN' : cause - Only 50 simultaneous API historical data requests allowed., contract: Stock(conId=56989171, symbol='SHREECEM', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='SHREECEM', tradingClass='SHREECEM')
ERROR:ib_insync.wrapper:Error 322, reqId 3001: Error processing request:-'bN' : cause - Only 50 simultaneous API h

ERROR:ib_insync.wrapper:Error 322, reqId 3026: Error processing request:-'bN' : cause - Only 50 simultaneous API historical data requests allowed., contract: Stock(conId=56983324, symbol='UBL', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='UBL', tradingClass='UBL')
ERROR:ib_insync.wrapper:Error 322, reqId 3027: Error processing request:-'bN' : cause - Only 50 simultaneous API historical data requests allowed., contract: Stock(conId=233076295, symbol='UJJIVAN', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='UJJIVAN', tradingClass='UJJIVAN')
ERROR:ib_insync.wrapper:Error 322, reqId 3028: Error processing request:-'bN' : cause - Only 50 simultaneous API historical data requests allowed., contract: Stock(conId=56983383, symbol='ULTRACEMC', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='ULTRACEMCO', tradingClass='ULTRACEMC')
ERROR:ib_insync.wrapper:Error 322, reqId 3029: Error processing request:-'bN' : cause - Only 50 simultan

In [4]:
optprice

[('ACC', 1480.0),
 ('ADANIENT', 156.0),
 ('ADANIPORT', 374.85),
 ('ADANIPOWE', 52.7),
 ('AJANTPHAR', 1091.1),
 ('ALBK', 46.05),
 ('AMARAJABA', 735.0),
 ('AMBUJACEM', 215.55),
 ('APOLLOHOS', 1213.0),
 ('APOLLOTYR', 238.0),
 ('ARVIND', 102.3),
 ('ASHOKLEY', 105.25),
 ('ASIANPAIN', 1329.25),
 ('AUROPHARM', 740.85),
 ('AXISBANK', 622.0),
 ('BAJAJ-AUT', 2839.25),
 ('BAJAJFINS', 6001.0),
 ('BAJFINANC', 2485.4),
 ('BALKRISIN', 933.0),
 ('BANKBAROD', 113.0),
 ('BANKINDIA', 85.05),
 ('BATAINDIA', 1080.5),
 ('BEL', 80.05),
 ('BEML', 748.0),
 ('BERGEPAIN', 317.5),
 ('BHARATFIN', 994.0),
 ('BHARATFOR', 518.0),
 ('BHARTIART', 311.0),
 ('BHEL', 66.35),
 ('BIOCON', 638.05),
 ('BOSCHLTD', 19170.0),
 ('BPCL', 328.4),
 ('BRITANNIA', 3058.0),
 ('CADILAHC', 353.05),
 ('CANBK', 257.8),
 ('CANFINHOM', 253.8),
 ('CAPF', 519.45),
 ('CASTROLIN', 148.6),
 ('CEATLTD', 1313.0),
 ('CENTURYTE', 910.0),
 ('CESC', 691.5),
 ('CGPOWER', 39.2),
 ('CHENNPETR', 267.85),
 ('CHOLAFIN', 1234.0),
 ('CIPLA', 532.05),
 ('COALIN

In [40]:
for i in range(0, len(contracts), blk):
    for c in contracts[i:i+blk]:
        print(c.symbol, ',', utils.getprice(contract=c, ib=ib))
#         ib.sleep(0.1)

AttributeError: 'list' object has no attribute 'reqId'

In [ ]:
utils.getprice(contract=contracts[25], ib=ib)

In [44]:
help(ib.cancelHistoricalData)

Help on method cancelHistoricalData in module ib_insync.ib:

cancelHistoricalData(bars:ib_insync.objects.BarDataList) -> None method of ib_insync.ib.IB instance
    Cancel the update subscription for the historical bars.



In [39]:
ib.disconnect()
while ib.isConnected():
    ib.sleep(3)
from ib_insync import *
util.startLoop()
ib = IB().connect('127.0.0.1', 7496, clientId=5) # kavi tws live
import importlib
importlib.reload(utils)

<module 'utils' from 'C:\\Users\\User\\Documents\\Scripts\\IB\\python\\lab\\utils.py'>

In [ ]:
df_q.contract.apply(get_price)

In [ ]:
#***   Error catching for list comprehension ***
import numpy as np
def catch(func, handle=lambda e : e, *args, **kwargs):
    '''List comprehension error catcher'''
    try:
        return func(*args, **kwargs)
    except Exception as e:
        np.nan
df_q['closeprice'] = [catch(lambda: getprice(c)) for c in df_q.contract]

In [ ]:
get_price(df_q.contract[3])

In [ ]:
df_q['closeprice'] = [get_price(c) for c in df_q.contract]

In [ ]:
# ***** Get list with Mlots + Margin% from 5paisa
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
df_paisa = pd.read_html(paisaurl, header=0)[1].drop_duplicates(subset='Symbol')

df_paisa1 = df_paisa[['Symbol', 'Mlot', 'TotMgn%']].sort_values('Symbol').reset_index(drop=True)

# Rename Symbol and Margin fields
df_paisa1 = df_paisa1.rename(columns={'Symbol': 'nseSymbol', 'TotMgn%': 'marginpct'})

# Convert columns to numeric and make margin to pct
df_paisa1 = df_paisa1.apply(pd.to_numeric, errors='ignore')
df_paisa1.marginpct = df_paisa1.marginpct.div(100)

In [ ]:
#***    make option chains and qualify   ***
chains = {s: ib.reqSecDefOptParams(s.symbol, '', s.secType, s.conId) for s in symbols}
chains

df_und = util.df(chains) # dataframe of underlying chains

tup_chains = [([k.symbol], tuple(i.expirations), tuple(i.strikes), ['P']) 
 for k, v in chains.items() 
 for i in v]

# create options without USDINR scrip
options = [Option(p[0], p[1], p[2], p[3], 'NSE') 
           for tup in tup_chains
           for p in product(*tup)]

# qualify the options
nse_qo = [t for i in range(0, len(options), 25) for t in ib.qualifyContracts(*options[i: i+25])]

df_qo = util.df(nse_qo) # dataframe for qualified options

In [ ]:
contract = nse_qo[0]
order = Order(action='SELL', orderType='MKT', totalQuantity=20)
ib.whatIfOrder(contract, order)

In [ ]:
ib.reqSecDefOptParams(futFopExchange='NSE', underlyingConId=337009725, underlyingSecType='IND', underlyingSymbol = 'BANKNIFTY')

In [ ]:
ib.disconnect()

# To-do
* something is wrong in the qual_options. Fix it.
* integration with standard deviation
* remove unnecessary options (Put)
* integration with black scholes
* integration with lot and margin
* generate orders
* place orders (what-if) to check margins (duplicate)
* check latest prices
* screen most valueable RoM and PoP
* place orders